# Intro to Data 

In [1]:
#import the libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('../regression_data.xls')
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7129300520,2014-10-13,3,1.00,1180,5650,1.0,0,0,3,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,6414100192,2014-12-09,3,2.25,2570,7242,2.0,0,0,3,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,5631500400,2015-02-25,2,1.00,770,10000,1.0,0,0,3,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2487200875,2014-12-09,4,3.00,1960,5000,1.0,0,0,5,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,1954400510,2015-02-18,3,2.00,1680,8080,1.0,0,0,3,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000


# Cleaning

* Drop irrelevant columns

In [3]:
df.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price'],
      dtype='object')

*id, date* - play no role in analysis

*sqft_above* - is (sqft_living - sqft_basement), hence can be dropped (avoid extra columns)

*sqft_living15, sqft_lot15* - show mean sqft of 15 neighboors (will not be used)

In [4]:
#save every change to another dataframe (not to lose data)
df1 = df.drop(['id', 'date', 'sqft_above', 'sqft_living15', 'sqft_lot15'], axis = 1)

* Check the shape, null values and data types

In [5]:
df1.info()  #we have no null values and only numerical data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       21597 non-null  int64  
 1   bathrooms      21597 non-null  float64
 2   sqft_living    21597 non-null  int64  
 3   sqft_lot       21597 non-null  int64  
 4   floors         21597 non-null  float64
 5   waterfront     21597 non-null  int64  
 6   view           21597 non-null  int64  
 7   condition      21597 non-null  int64  
 8   grade          21597 non-null  int64  
 9   sqft_basement  21597 non-null  int64  
 10  yr_built       21597 non-null  int64  
 11  yr_renovated   21597 non-null  int64  
 12  zipcode        21597 non-null  int64  
 13  lat            21597 non-null  float64
 14  long           21597 non-null  float64
 15  price          21597 non-null  int64  
dtypes: float64(4), int64(12)
memory usage: 2.6 MB


* Check unique values 

In [6]:
for col in df1:
    print('-----------------------')
    print(f'{col.upper()} has {df1[col].nunique()} unique values:')
    print(df1[col].value_counts())
    print()

-----------------------
BEDROOMS has 12 unique values:
3     9824
4     6882
2     2760
5     1601
6      272
1      196
7       38
8       13
9        6
10       3
11       1
33       1
Name: bedrooms, dtype: int64

-----------------------
BATHROOMS has 29 unique values:
2.50    5377
1.00    3851
1.75    3048
2.25    2047
2.00    1930
1.50    1445
2.75    1185
3.00     753
3.50     731
3.25     589
3.75     155
4.00     136
4.50     100
4.25      79
0.75      71
4.75      23
5.00      21
5.25      13
5.50      10
1.25       9
6.00       6
0.50       4
5.75       4
6.75       2
8.00       2
6.25       2
6.50       2
7.50       1
7.75       1
Name: bathrooms, dtype: int64

-----------------------
SQFT_LIVING has 1034 unique values:
1300    138
1400    135
1440    133
1800    129
1660    129
       ... 
2507      1
9890      1
5584      1
1961      1
1425      1
Name: sqft_living, Length: 1034, dtype: int64

-----------------------
SQFT_LOT has 9776 unique values:
5000     358
6000     2

* Clean the columns (rename and replace)

*sqft_basement* - sqft of basement is not so important, as more than half of the properties don't have a basement. Change it to column 'basement' with values 0 (if there is no) and 1 (if there is any).

*yr_renovated* - the same with basement (even more not renovated, > 90%). Change it to column 'renovated' (values 0, 1).

In [7]:
#for changes made create another dataframe
df2 = df1

In [8]:
for col in df2:
    if col in ('sqft_basement', 'yr_renovated'):
        df2[col] = df2[col].apply(lambda x: x if x == 0 else 1)
        df2 = df2.rename(columns = {col: col.split('_')[1]})

In [9]:
#check
df2.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,basement,yr_built,renovated,zipcode,lat,long,price
0,3,1.00,1180,5650,1.0,0,0,3,7,0,1955,0,98178,47.5112,-122.257,221900
1,3,2.25,2570,7242,2.0,0,0,3,7,1,1951,1,98125,47.7210,-122.319,538000
2,2,1.00,770,10000,1.0,0,0,3,6,0,1933,0,98028,47.7379,-122.233,180000
3,4,3.00,1960,5000,1.0,0,0,5,7,1,1965,0,98136,47.5208,-122.393,604000
4,3,2.00,1680,8080,1.0,0,0,3,8,0,1987,0,98074,47.6168,-122.045,510000


Columns *zipcode, lat and long* will remain for visualization in Tableau, but not be used for further prediction.

Columns *waterfront, view, condition, grade, basement and renovated* are categorical in nature. They will only be changed in Tableau, as for prediction we use numbers.

In [10]:
#save the new dataset
df2.to_excel('cleaned_data.xls', index = False)